In [1]:
import pandas as pd
import numpy as np

from pytrends.request import TrendReq

from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import Label, ColumnDataSource, Select, DatetimeTickFormatter
from bokeh.models import RangeTool, NumeralTickFormatter, HoverTool, Legend
from bokeh.models.tools import BoxZoomTool, ResetTool
from bokeh.io import output_notebook, show

output_notebook()

Loading BokehJS ...

In [2]:
pytrends = TrendReq()

# keywords = ['python', 'php']
geo = 'ID'
gprop = 'youtube'

timeframes = [
    'today 5-y',
    'today 12-m',
    'today 3-m', 
    'today 1-m',
    'now 1-d',
    'now 7-d',
    'now 1-H',
    'now 4-H']

# pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=timeframes[4])

In [3]:
def fetch_data(keywords, geo, gprop, timeframe):
    d = {}
    for i in range(len(keywords)):
        pytrends.build_payload([keywords[i]], geo=geo, gprop=gprop, timeframe=timeframe)
        data = pytrends.interest_over_time()
        data.reset_index(inplace=True)
        data.rename(columns={'date':'datetime', keywords[i]: 'value', 'isPartial':'partial'}, inplace=True)
        d[i] = dict(x=data.datetime, y=data.value)
    return d

In [4]:
keywords = ['php', 'python']
timeframe = timeframes[3]
data = fetch_data(keywords, geo, gprop, timeframe)

In [5]:
x = [data[i]['x'] for i, x in enumerate(data)]
y = [data[i]['y'] for i, x in enumerate(data)]
z = [round(np.array(data[i]['y']).mean(), 2) for i, x in enumerate(data)]

In [6]:
# [(keywords[i], [z[i]]) for i, x in enumerate(z)]

In [7]:
import numpy as np
import random
from bokeh.plotting import figure, show

colors = ['#338CFF', '#FF4033']

# line graph
p = figure(
    width=800, 
    height=350, 
    x_axis_label="", 
    y_axis_label="", 
    toolbar_location=None, 
    title=f"Interest over time ({timeframe})")

def make_line(x):
    for i in range(len(x)):
        p.line(
            np.array(x)[i], 
            np.array(y)[i], 
            line_width=3, 
            alpha=0.8, 
            legend_label=keywords[i], 
            line_color=colors[i])
    
make_line(x)

p.xaxis[0].formatter = DatetimeTickFormatter(days=["%b %d, %Y"])
p.legend.location = "top_left"
p.xgrid.visible = False

# bar graph
b = figure(
    x_range=keywords, 
    width=200, 
    height=350,
    title="Average", 
    toolbar_location=None, 
    tools="")

b.vbar(
    x=keywords, 
    top=z, 
    width=0.9, 
    fill_color=colors)

b.xgrid.grid_line_color = None
b.y_range.start = 0

# layouts
layout = column(row(b, p))

# ploting
show(layout)

In [8]:
# from bokeh.plotting import figure
# from bokeh.io import show
# from bokeh.models import Legend
# from bokeh.palettes import brewer

# months = ['JAN', 'FEB', 'MAR']
# data = {"month" : months,
#         "cat1"  : [1, 4, 12],
#         "cat2"  : [2, 5, 3],
#         "cat3"  : [5, 6, 1],
#         "cat4"  : [8, 2, 1],
#         "cat5"  : [1, 1, 3]}
# categories = list(data.keys())
# categories.remove('month')

# colors = brewer['YlGnBu'][len(categories)]

# p = figure(x_range=months, plot_height=250, title="Categories by month",
#            toolbar_location=None)
# v = p.vbar_stack(categories, x='month', width=0.9, color=colors, source=data)

# legend = Legend(items=[(x, [v[i]]) for i, x in enumerate(categories)], location=(0, -30))

# p.add_layout(legend, 'right')

# show(p)

In [9]:
# from bokeh.models import CategoricalColorMapper, Legend
# from bokeh.palettes import Category10
# from bokeh.plotting import figure, show
# from bokeh.sampledata.iris import flowers

# color_mapper = CategoricalColorMapper(
#     factors=[x for x in flowers['species'].unique()], 
#     palette=Category10[10])

# p = figure(height=350, width=500)
# p.add_layout(Legend(), 'right')
# p.circle(
#     "petal_length", 
#     "petal_width", 
#     source=flowers, 
#     legend_group='species', 
#     color=dict(
#         field='species', 
#         transform=color_mapper))

# show(p)